# IS 4487 Lab 3

## Outline

Repeat exercises from Lab 2, but with the *SuperStore Retail Orders* dataset
- Loading installed packages
- Inspect data
- Explore plot types

<a href="https://colab.research.google.com/github/Stan-Pugsley/is_4487_base/blob/main/Labs/Scripts/lab_03_retailer_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Superstore Data Dictionary

 ORDER VARIABLES:
 - Order ID
 - Order Date
 - Order Year Month - Year and month of the order
 - Order Type - Was the order completed at a store or online? (Retail, Online)
 - Quantity - Quantity ordered for the product

 CUSTOMER VARIABLES:
 - Customer Name
 - City
 - State Province
 - Email

PRODUCT VARIABLES:
 - Product Name
 - Product Line - Category of the product (i.e. Bikes Phones)
 - Product Price - Price in US Dollars
 - Product Status - Current status of the product (Active, Inactive)

## Load Libraries

➡️ Assignment Tasks
- Load any necessary libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


## Import Data into Dataframe

➡️ Assignment Tasks
- Import data from the SuperStore retail dataset into a dataframe (in GitHub go to Labs > DataSets)
- Describe or profile the dataframe

In [3]:
url = 'https://github.com/Stan-Pugsley/is_4487_base/blob/main/Labs/DataSets/superstore_retail_orders.csv?raw=true'
df = pd.read_csv(url, on_bad_lines='skip')
print(df.head())
print(df.info())
print(df.describe())

   order_id  order_date order_year_month  customer_name               city  \
0     51188  2023-07-01          2023-07  Audrey Blanco  North Cherylmouth   
1     51187  2023-07-01          2023-07    Isaac Allen         Traceyland   
2     51199  2023-07-02          2023-07     Ian Morgan          Dylanberg   
3     51241  2023-07-04          2023-07     Alexa Bell     Lake Tracytown   
4     51297  2023-07-08          2023-07     Casey Diaz        Byrdchester   

  state_province                    email order_type  quantity  \
0  New Hampshire  audrey_blanco@email.com     Retail       3.0   
1  West Virginia    isaac_allen@email.com     Online       4.0   
2   South Dakota     ian_morgan@email.com     Online       5.0   
3      Wisconsin     alexa_bell@email.com     Online       1.0   
4    Mississippi     casey_diaz@email.com     Online       1.0   

                                        product_name product_line  \
0  Apple iPhone XR (Yellow, 256 GB) (Includes Ear...       Mobile

## Prepare Data

➡️ Assignment Tasks
- Create a "revenue" variable and make sure it is a numeric datatype
- Remove rows with empty (NULL) values
- Locate outliers in at least two of the variables.   Filter them out if they are mistakes; keep them if they are exceptional cases.

In [10]:
# Create the 'revenue' variable by multiplying 'quantity' and 'product_price'
df['revenue'] = df['quantity'] * df['product_price']

# Ensure 'revenue' is numeric
df['revenue'] = pd.to_numeric(df['revenue'], errors='coerce')

# Verify the new column
print(df[['quantity', 'product_price', 'revenue']].head())
print("Data type of 'revenue':", df['revenue'].dtype)


   quantity  product_price  revenue
0       3.0         649.99  1949.97
1       4.0         649.99  2599.96
2       5.0         649.99  3249.95
3       1.0         649.99   649.99
4       1.0         649.99   649.99
Data type of 'revenue': float64


In [11]:
# Remove rows with NULL values
df = df.dropna()

# Verify there are no NULL values left
print("Number of NULL values after dropping rows:")
print(df.isnull().sum())

# Display the shape of the cleaned dataset
print(f"Shape of the dataset after removing NULL values: {df.shape}")


Number of NULL values after dropping rows:
order_id            0
order_date          0
order_year_month    0
customer_name       0
city                0
state_province      0
email               0
order_type          0
quantity            0
product_name        0
product_line        0
product_price       0
product_status      0
revenue             0
dtype: int64
Shape of the dataset after removing NULL values: (55498, 14)


In [15]:
# Function to detect outliers using IQR
def detect_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)  # First quartile (25th percentile)
    Q3 = data[column].quantile(0.75)  # Third quartile (75th percentile)
    IQR = Q3 - Q1  # Interquartile Range
    lower_bound = Q1 - 1.5 * IQR  # Lower bound for outliers
    upper_bound = Q3 + 1.5 * IQR  # Upper bound for outliers
    return data[(data[column] < lower_bound) | (data[column] > upper_bound)], lower_bound, upper_bound

# Detect and print outliers for 'quantity'
outliers_quantity, lower_quantity, upper_quantity = detect_outliers_iqr(df, 'quantity')
print("Outliers in Quantity:")
print(outliers_quantity)

# Detect and print outliers for 'product_price'
outliers_price, lower_price, upper_price = detect_outliers_iqr(df, 'product_price')
print("Outliers in Product Price:")
print(outliers_price)

# Filter out mistakes (optional based on domain knowledge)
df_cleaned = df[
    (df['quantity'] >= lower_quantity) & (df['quantity'] <= upper_quantity) &
    (df['product_price'] >= lower_price) & (df['product_price'] <= upper_price)
]

# Keep exceptional cases separately for review
exceptional_cases = df[
    (df['quantity'] > upper_quantity) |
    (df['product_price'] > upper_price)
]

print(f"Cleaned Dataset Shape: {df_cleaned.shape}")
print("Exceptional Cases (Kept for Review):")
print(exceptional_cases)


Outliers in Quantity:
       order_id  order_date order_year_month     customer_name  \
1         51187  2023-07-01          2023-07       Isaac Allen   
2         51199  2023-07-02          2023-07        Ian Morgan   
33727     66374  2024-03-18          2024-03     David Jenkins   
33822     66829  2024-03-25          2024-03     Destiny Smith   
44626     45087  2022-01-03          2022-01     Joseph Martin   
45530     45086  2022-01-02          2022-01    Jacquelyn Diaz   
45752     45706  2022-03-20          2022-03        David Diaz   
45857     45085  2022-01-02          2022-01  Reginald Alvarez   
55308     66470  2024-03-19          2024-03         Lisa Wang   

                    city state_province                       email  \
1             Traceyland  West Virginia       isaac_allen@email.com   
2              Dylanberg   South Dakota        ian_morgan@email.com   
33727     Port Christina          Idaho     david_jenkins@email.com   
33822  New Hannahborough        I

## Summarize Data

➡️ Assignment Tasks
- Display the average price per product line
- Display the total revenue for all months

In [12]:
#average price
# Group by 'product_line' and calculate the average price
average_price_per_product_line = df.groupby('product_line')['product_price'].mean()

# Display the result
print("Average Price per Product Line:")
print(average_price_per_product_line)


Average Price per Product Line:
product_line
Accessories          11.531647
Bikes              1821.807035
Cameras             779.174673
Clothing          48943.949314
Electronics         693.973186
Laptop             3578.270000
Mobile             1013.581162
OfficeProducts        8.640000
Projector          3386.564074
Name: product_price, dtype: float64


In [13]:
#total revenue
# Ensure the revenue column exists
if 'revenue' not in df.columns:
    df['revenue'] = df['quantity'] * df['product_price']

# Group by 'order_year_month' and calculate the total revenue
total_revenue_per_month = df.groupby('order_year_month')['revenue'].sum()

# Display the result
print("Total Revenue for All Months:")
print(total_revenue_per_month)


Total Revenue for All Months:
order_year_month
1922-06      979636.00
2022-01     4746737.16
2022-02      500428.41
2022-03      708048.28
2022-04      653364.08
2022-05      659325.94
2022-06      637784.29
2022-07      486114.93
2022-08      533572.11
2022-09      341266.49
2022-10      404276.65
2022-11      306453.60
2022-12      560965.27
2023-01      432425.86
2023-02      459752.28
2023-03      469912.92
2023-04      492886.16
2023-05      539365.40
2023-06      529704.69
2023-07    10495300.14
2023-08    47232524.31
2023-09    50614963.75
2023-10    53117217.85
2023-11    50196883.14
2023-12    59182870.87
2024-01    57744443.81
2024-02    57700422.24
2024-03    57460241.29
2024-04    63213208.20
2024-05    73469869.94
2024-06    67964592.02
2029-06          19.55
2029-07      517505.48
2029-08          39.94
2421-09       32051.29
Name: revenue, dtype: float64
